In [1]:
import numpy as np
import cv2
import os
import glob
from enum import Enum

In [2]:
os.chdir('./data/')

In [3]:
class DrawOption(Enum):
    AXES = 1
    CUBE = 2

In [57]:
def draw_axis(img, corners, img_pts):
    def tuple_of_int(arr):
        return tuple(int(i) for i in arr)
    
    corner = tuple_of_int(corners[0].ravel())
    # print(corners[0].ravel()) -> corner coordinates for every image
    # print(img_pts[0].ravel()) -> 3 points for every image corner 
    # print('#'*50)

    # draw the axis
    img = cv2.line(img, corner, tuple_of_int(img_pts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple_of_int(img_pts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple_of_int(img_pts[2].ravel()), (0,0,255), 5)
    
    return img

def draw_cube(img, img_pts):
    img_pts = np.int32(img_pts).reshape(-1,2)
    # print(img_pts) -> 8 corner points
    # print('#'*50)
    
    # draw ground floor in green
    '''
    img_pts[:4] ->[598  67][453  78][472 251][628 233] 4 point that form the ground floor
    img_pts[4:] ->[617 -42][446 -41][472 152][665 136] 4 point that form the top floor
    '''
    img = cv2.drawContours(img, [img_pts[:4]],-1,(0,255,0),-1)
    
    # draw pillars in blue color
    for i in range(4):
        j = i+4
        # draw line from every point in ground floor to top floor
        img = cv2.line(img, tuple(img_pts[i]), tuple(img_pts[j]), (255,0,0), 3)

        # draw top layer in red color
        img = cv2.drawContours(img, [img_pts[4:]],-1,(0,0,255), 3)
    
    return img

In [49]:
def pose_estimation(option: DrawOption):
    data = np.load('./calibration_data/MultiMatrix.npz')

    cam_matrix = data['cam_matrix']
    dist_matrix = data['dist_coeffs']

    img_paths = glob.glob('./chess_board_images/*.png')

    num_rows = 7
    num_cols = 7

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    obj_3D = np.zeros((num_rows * num_cols, 3), np.float32)
    obj_3D[:, :2] = np.mgrid[0:num_rows, 0:num_cols].T.reshape(-1, 2)

    # real world points of objects
    axis = np.float32([[2,0,0],[0,2,0],[0,0,-2]])
    cubeCorners = np.float32([[0,0,0],[0,2,0],[2,2,0],[2,0,0],[0,0,-2],[0,2,-2],[2,2,-2],[2,0,-2]])

    for img_path in img_paths:
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (num_rows, num_cols), None)

        if ret:
            improve_corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            ret, rvecs, tvecs = cv2.solvePnP(obj_3D, improve_corners, cam_matrix, dist_matrix) # to estimate the pose (rotation and translation vectors) of the chessboard in the image

            if option == DrawOption.AXES:
                # projection 3D point on 2D image
                img_pts,_ = cv2.projectPoints(axis, rvecs, tvecs, cam_matrix, dist_matrix)
                # print(img_pts)
                img = draw_axis(img, improve_corners, img_pts)
            
            if option == DrawOption.CUBE:
                # project on 2d image
                img_pts,_ = cv2.projectPoints(cubeCorners, rvecs, tvecs, cam_matrix, dist_matrix)
                img = draw_cube(img, img_pts)
            
            cv2.imshow('img', img)
            cv2.waitKey(5000)
    
    cv2.destroyAllWindows()

In [58]:
if __name__ == '__main__':
    # pose_estimation(DrawOption.AXES)
    pose_estimation(DrawOption.CUBE)